In [2]:
import requests
import json
import datetime
import boto3
from decimal import Decimal
from requests_aws4auth import AWS4Auth

In [2]:
MY_API_KEY = ""
headers = {'Authorization': 'Bearer %s' % MY_API_KEY}
URL = "https://api.yelp.com/v3/businesses/search"
LIMIT = 30
HOST = "https://search-dinningconcierge-7jazrt3icqbzgkc35r4n5tdvfm.us-east-1.es.amazonaws.com"
region = 'us-east-1'
service = 'es'
HOST = "https://search-dinningconcierge-7jazrt3icqbzgkc35r4n5tdvfm.us-east-1.es.amazonaws.com/"
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service, session_token=credentials.token)

In [3]:
def get_total(cuisine, location):
    params = {
        'term': cuisine+' restaurants', 
        'location': location,
    }
    req = requests.get(URL, params = params, headers=headers)
    if req.status_code==200:
        return json.loads(req.text)['total']
    return 0

In [4]:
def search(cuisine, location, offset):
    params = {
        'term': cuisine+' restaurants', 
        'location': location, 
        'limit': LIMIT, 
        'offset': offset
    }
    return requests.get(URL, params = params, headers=headers)

In [5]:
def formatting(business, cuisine):
    return json.loads(json.dumps({
        "id": business['id'],
        "name": business['name'],
        "address": ", ".join(business['location']['display_address']),
        "coordinates": business['coordinates'],
        "review_cnt": business['review_count'],
        "rating": business['rating'],
        "zip_code": business['location']['zip_code'],
        "cuisine": cuisine,
    }), parse_float=Decimal)

In [6]:
def upload_yelp(businesses):
    dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
    table = dynamodb.Table('yelp-restaurants')
    for business in businesses:
        business["insertedAtTimestamp"] = datetime.datetime.now().strftime("%m/%d/%Y %H:%M:%S")
        table.put_item(Item = business)

In [ ]:
def upload_openSearch(businesses):
    for business in businesses:
        body = {
        "id": business['id'],
        "cuisine": business['cuisine']
        }
        query = HOST + '/{_index}/{_type}/{_id}'.format(_index="restaurants", _type="Restaurant", _id=business['id'])
        req = requests.put(query, auth=awsauth, data=json.dumps(body), headers = {'Content-type': 'application/json'})

In [11]:
cuisine_types = ['American', "Chinese", "Indian", "Italian", "Korean"]
locations = ['Manhattan', 'Queens', 'Brooklyn', 'Bronx', 'Staten Island']
for cuisine in cuisine_types:
    res = []
    for location in locations:
        total = get_total(cuisine, location)
        print(cuisine, location, total)
        offset = 0
        while(offset<=total-LIMIT):
            req = search(cuisine, location, offset)
            businesses = json.loads(req.text)['businesses']
            for business in businesses:
                bus = formatting(business, cuisine)
                if bus not in res:
                    res.append(bus)
            offset += LIMIT
    upload_yelp(res)
    upload_openSearch(res)

Korean Manhattan 240
Korean Queens 240
Korean Brooklyn 240
Korean Bronx 240
Korean Staten Island 240
